In [1]:
# NumPyをインポート
import numpy as np

# SciPy積分パッケージをインポート
from scipy import integrate

import time
import math
import torch
from scipy.stats import norm  # 正規分布

### $\int_{-1}^1 |x|dx$

In [2]:
# 関数を定義
y = lambda x:np.abs(x)
# yを0からpiまで数値積分
ie = integrate.quad(y, -1,1)

print(ie[0])#[1]成分は誤差

1.0


### $\int_0^1\frac{1}{x^2+a^2}dx$, $a=2$

In [3]:
# 関数を定義
y = lambda x, a: 1/(x**2 + a**2)

# a=2として、yを0から1まで積分
iy, err = integrate.quad(y, 0, 1, args=2)

print(iy)

0.23182380450040307


### $n\geq 2$の時に$J_n = \int_a^bx^ne^{-\frac{x^2}{2}}dx$の計算が、直接積分するか再起関数を作るかどっちが早いか確かめる

$n=50, a=0, b=1$でやってみる

In [4]:
n=2
a=-4
b=np.inf

まずは直接計算

In [5]:
y = lambda x, n: x**n*np.exp(-x**2/2)

start=time.time()
iy, err = integrate.quad(y, a, b, args=n)
end=time.time()

print(iy)
print("計算時間：",end-start)

2.5052070360891188
計算時間： 0.0010712146759033203


次は再起的に

In [6]:
def torchJn(n, a, b):
    if n==0:
        return torch.sqrt(torch.tensor(np.pi/2))*( torch.erf(b/np.sqrt(2)) - torch.erf(a/np.sqrt(2)) )
    elif n==1:
        return torch.exp(-a**2/2.0)-torch.exp(-b**2/2.0)
    else:
        ret = 0
        if not np.abs(a.item()) == np.inf:
            ret += a**(n-1)*torch.exp(-a**2/2.0)
        if not np.abs(b.item()) == np.inf:
            ret -= b**(n-1)*torch.exp(-b**2/2.0)
        ret += (n-1)*torchJn(n-2, a, b)
        return ret

In [7]:
start=time.time()
iy = torchJn(n, torch.tensor(a), torch.tensor(b))
end=time.time()
print(iy.item())
print("計算時間：",end-start)

2.505207061767578
計算時間： 0.0033409595489501953


### $p> 0$の時に$J_n =\frac{1}{\sqrt{2\pi}} \int_a^b|x-c|^pe^{-\frac{x^2}{2}}dx$の計算が、直接積分するか再起関数を作るかどっちが早いか確かめる

In [15]:
p=2
a= - np.inf
b=0.9
c=0.5

In [16]:
y = lambda x, p: np.abs(x-c)**p*np.exp(-x**2/2)/np.sqrt(2*np.pi)

start=time.time()
iy, err = integrate.quad(y, a, b, args=p)
end=time.time()

print(iy)
print("計算時間：",end-start)

1.0465333683064262
計算時間： 0.003323793411254883


In [17]:
def combinations_count(n, r):
    return math.factorial(n) // (math.factorial(n - r) * math.factorial(r))
def integral(a, b, c, p):
    ret=0
    if a>=c:
        for i in range(p+1):
            ret += combinations_count(p,i)*(-c)**i/np.sqrt(2*np.pi)*torchJn(n=p-i, a=a, b=b)
    elif a<c and c<b:
        ret = integral(a=a, b=c, c=c, p=p) + integral(a=c, b=b, c=c, p=p)
    else:
        for i in range(p+1):
            ret += combinations_count(p,i)*c**i*(-1)**(p-i)/np.sqrt(2*np.pi)*torchJn(n=p-i, a=a, b=b)
    return ret

In [18]:
integral(a=torch.tensor(a), b=torch.tensor(b), c=torch.tensor(c), p=p).item()

start=time.time()
iy = integral(a=torch.tensor(a), b=torch.tensor(b), c=torch.tensor(c), p=p).item()
end=time.time()

print(iy)
print("計算時間：",end-start)

1.0465333461761475
計算時間： 0.0017223358154296875


ま結局再起関数の方でやるのかね

### $W_p=\left\{\sum_{n=1}^N\int_{\Phi^{-1}(\frac{n-1}{N})}^{\Phi^{-1}(\frac{n}{N})}|x_n-x|^pf_Q(x)dx\right\}^{\frac{1}{p}}$

In [19]:
# 標準正規分布の累積分布関数の逆関数
torch.tensor(norm.ppf(q=0.1, loc=0, scale=1))

tensor(-1.2816)

In [20]:
def pWasserstein(x, p):
    N=x.shape[0]
    ret = 0
    for n in range(1,N+1):
        ret += integral(a=torch.tensor(norm.ppf(q=(n-1)/N, loc=0, scale=1)), b=torch.tensor(norm.ppf(q=n/N, loc=0, scale=1)),c=x[n-1],p=p)
    return ret

In [21]:
x=torch.tensor([0.01,0.02,0.04,0.05])
pWasserstein(x, p=2)

tensor(0.9725)